<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/Poczenie-baz-danych-nagrd-z-main-data/colabs/Wczytanie_danych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup sparka

# Załadowanie nowych baz

##Setup sparka

In [24]:
!pip install pyspark py4j
!pip install -q findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
!tar xf spark-3.3.1-bin-hadoop2.tgz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-11-27 13:06:29--  ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
           => ‘spark-3.3.1-bin-hadoop2.tgz.1’
Resolving ftp.ps.pl (ftp.ps.pl)... 212.14.28.36, 2001:4c58:62:2::36
Connecting to ftp.ps.pl (ftp.ps.pl)|212.14.28.36|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/apache/spark/spark-3.3.1 ... done.
==> SIZE spark-3.3.1-bin-hadoop2.tgz ... 274099817
==> PASV ... done.    ==> RETR spark-3.3.1-bin-hadoop2.tgz ... done.
Length: 274099817 (261M) (unauthoritative)

spark-3.3.1-bin-had 100%[===================>] 261.40M  23.0MB/s    in 12s     

2022-11-27 13:06:44 (21.4 MB/s) - ‘spark-3.3.1-bin-hadoop2.tgz.1’ saved [274099817]



In [25]:
import pyspark
import findspark
from pyspark.sql import SparkSession
import os

In [26]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"
spark=SparkSession.builder.appName('Colab').getOrCreate()
spark

#Pobranie danych

##Import danych

In [27]:
!wget https://datasets.imdbws.com/name.basics.tsv.gz
!wget https://datasets.imdbws.com/title.akas.tsv.gz
!wget https://datasets.imdbws.com/title.basics.tsv.gz
!wget https://datasets.imdbws.com/title.crew.tsv.gz
!wget https://datasets.imdbws.com/title.episode.tsv.gz
!wget https://datasets.imdbws.com/title.principals.tsv.gz
!wget https://datasets.imdbws.com/title.ratings.tsv.gz

--2022-11-27 13:06:50--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 18.165.32.119, 18.165.32.20, 18.165.32.74, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|18.165.32.119|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 236951880 (226M) [binary/octet-stream]
Saving to: ‘name.basics.tsv.gz.1’

name.basics.tsv.gz. 100%[===================>] 225.97M  37.6MB/s    in 5.5s    

2022-11-27 13:06:57 (41.0 MB/s) - ‘name.basics.tsv.gz.1’ saved [236951880/236951880]

--2022-11-27 13:06:57--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.249.98.35, 13.249.98.61, 13.249.98.91, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.249.98.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 288688343 (275M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz.1’

title.akas.tsv.gz.1 100%[===================>] 275.31

##Rozpakowanie danych

In [28]:
!gzip -dc /content/name.basics.tsv.gz > name.basics.csv
!gzip -dc /content/title.akas.tsv.gz > title.akas.csv
!gzip -dc /content/title.basics.tsv.gz > title.basic.csv
!gzip -dc /content/title.crew.tsv.gz > title.crew.csv
!gzip -dc /content/title.episode.tsv.gz > title.episode.csv
!gzip -dc /content/title.principals.tsv.gz > title.principals.csv
!gzip -dc /content/title.ratings.tsv.gz > title.ratings.csv

#Wczytanie danych

##Wczytajmy dane z rozpakowanych plików

In [29]:
df_name_basics=spark.read.option("header", "true").option("delimiter", "\t").csv('name.basics.csv' ) 
df_title_akas=spark.read.option("header","true").option("delimiter", "\t").csv('title.akas.csv')
df_title_basic=spark.read.option("header","true").option("delimiter", "\t").csv('title.basic.csv')
df_title_crew=spark.read.option("header","true").option("delimiter", "\t").csv('title.crew.csv')
df_title_episode=spark.read.option("header","true").option("delimiter","\t").csv('title.episode.csv')
df_title_principals=spark.read.option("header","true").option("delimiter","\t").csv('title.principals.csv')
df_title_ratings=spark.read.option("header","true").option("delimiter","\t").csv('title.ratings.csv')

##Wyświetlmy dane

In [30]:
df_name_basics.show(3)
df_title_akas.show(3)
df_title_basic.show(3)
df_title_crew.show(3)
df_title_episode.show(3)
df_title_principals.show(3)
df_title_ratings.show(3)

+---------+---------------+---------+---------+--------------------+--------------------+
|   nconst|    primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+---------------+---------+---------+--------------------+--------------------+
|nm0000001|   Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0031983,tt00504...|
|nm0000002|  Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0038355,tt00373...|
|nm0000003|Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0054452,tt00573...|
+---------+---------------+---------+---------+--------------------+--------------------+
only showing top 3 rows

+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|  titleId|ordering|               title|region|language|      types|   attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|tt0000001|       1|          Карменсіта|    

# Złączenie istotnych kolumn

## Wybierzmy z tabel tylko istotne rekordy:
- df_title_basic: wybierzmy tylko rekordy z primaryProfession zawierającym "actor" lub "actress",
- df_title_principals: wybierzmy rekordy z category zawierającym "actor" lub "actress".

In [31]:
df_name_basics_selected = df_name_basics.filter("primaryProfession like '%actor%' or primaryProfession like '%actress%'")
df_title_principals_selected = df_title_principals.filter((df_title_principals.category == "actor") | (df_title_principals.category == "actress"))

## Wybierzmy następujące kolumny z tabel:
- df_title_basic:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - titleType (rodzaj tytułu, np. film, serial - można porównywać podobieństwo aktorów na podstawie tego w jakich typach produkcji grali)
  - originalTitle (tytuł produkcji - być może przyda się do analizy podobnych filmów)
  - isAdult (czy produkcja jest dla dorosłych - można wykorzystać do porównywania aktorów na podstawie tego czy grają głównie w produkcjach dla dorosłych)
  - startYear (data wypuszczenia produkcji - można wykorzystać do porównania epok filmów, w których grali aktorzy)
  - endYear (data zakończenia serii, w innych przypadkach \N - można wykorzystać do porównania epok filmów, w których grali aktorzy)
  - genres (gatunki - może posłużyć do porównania aktorów na podstawie tego, że grali w produkcjach o podobnych gatunkach)

- df_title_principals:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - nconst (unikalny numer aktora - potrzebne do złączenia tabel)
  - category (kategoria pracy (actor/actress) - może posłużyć do porównania aktorów na podstawie takiej samej płci)
  - characters (postacie zagrane w produkcji - może posłużyć do porównania aktorów na podstawie podobnych granych postaci)

- df_name_basics:
  - nconst(unikalny numer aktora - potrzebne do złączenia tabel)
  - primaryName (imię i nazwisko aktora - potrzebne do wyświetlania aktora lub do odnajdywania go w bazie po imieniu i nazwisku)
  - knownForTitles (tytuły produkcji, z których znany jest aktor - może posłużyć do porównania aktorów na podstawie tych samych lub podobnych produkcji, z których są znani)

In [32]:
df_title_basic_selected = df_title_basic.select(["tconst", "titleType", "originalTitle", "isAdult", "startYear", "endYear", "genres"])
df_title_principals_selected = df_title_principals_selected.select(["tconst", "nconst", "category", "characters"])
df_name_basics_selected = df_name_basics_selected.select(["nconst", "primaryName", "knownForTitles"])

# Złączmy tabele df_name_basics, df_title_principals i df_title_basic

In [99]:
# print("df_name_basics_selected dataframe size: ", (df_name_basics_selected.count(), len(df_name_basics_selected.columns)))
# print("df_title_principals_selected dataframe size: ", (df_title_principals_selected.count(), len(df_title_principals_selected.columns)))
# print("df_title_basic_selected dataframe size: ", (df_title_basic_selected.count(), len(df_title_basic_selected.columns)))
data = df_title_basic_selected.join(df_title_principals_selected, "tconst", "right")
# print("joined dataframe size: ", (data.count(), len(data.columns)))
data = data.join(df_name_basics_selected, "nconst", "inner")
# print("joined dataframe size: ", (data.count(), len(data.columns)))

## Łączymy dane na temat jednego aktora w jeden rekord

In [100]:
from pyspark.sql.functions import collect_list
from pyspark.sql.functions import first
from pyspark.sql.functions import min
from pyspark.sql.functions import max
data = data.groupby('nconst').agg(collect_list('tconst').alias("tconst"), collect_list('titleType').alias("titleType"), collect_list('originalTitle').alias("originalTitle"), collect_list('isAdult').alias("isAdult"), min('startYear').alias("startYear"), max('endYear').alias("endYear"), collect_list('genres').alias("genres"), first('category').alias("category"), collect_list('characters').alias("characters"), first('primaryName').alias("primaryName"), first('knownForTitles').alias("knownForTitles"))


In [94]:
data.show()

+---------+--------------------+--------------------+--------------------+--------------------+---------+-------+--------------------+--------+--------------------+--------------------+--------------------+
|   nconst|              tconst|           titleType|       originalTitle|             isAdult|startYear|endYear|              genres|category|          characters|         primaryName|      knownForTitles|
+---------+--------------------+--------------------+--------------------+--------------------+---------+-------+--------------------+--------+--------------------+--------------------+--------------------+
|nm0000004|[tt0694438, tt069...|[tvEpisode, tvEpi...|[Jill Clayburgh/L...|[0, 0, 0, 0, 0, 0...|     1973|     \N|[Comedy,Music, Co...|   actor|[["Jawaharlal Neh...|        John Belushi|tt0072562,tt00804...|
|nm0000012|[tt0058413, tt005...|[movie, movie, mo...|[La noia, The Nan...|[0, 0, 0, 0, 0, 0...|     1931|     \N|[Drama, Mystery,T...| actress|[["Dino's mother"...|        

# Wybierzmy przykładowe wiersze z końcowego dataframe'u

In [35]:
sample = data.rdd.takeSample(False, 5)

In [36]:
import pandas as pd
sample = pd.DataFrame(sample, columns = data.columns)
sample

,nconst,tconst,titleType,originalTitle,isAdult,startYear,endYear,genres,category,characters,primaryName,knownForTitles
0,nm11138476,tt11360104,tvEpisode,Episode #2.190,0,\N,\N,Romance,actress,\N,Rachitha Mahalakshmi,"tt11299474,tt12263646,tt12398416,tt13759830"
1,nm0256607,tt12090724,tvEpisode,Episode #17.81,0,\N,\N,"Action,Adventure,Animation",actress,"[""Tomomi""]",Hiroko Emori,"tt1173898,tt1547047,tt0161952,tt0142242"
2,nm0152082,tt1152271,video,Mexican American,0,2007,\N,Action,actor,"[""Tony""]",Damian Chapa,"tt0105690,tt0106469,tt1811376,tt7627774"
3,nm0000329,tt2992152,movie,Si ren ding zhi,0,2013,\N,"Comedy,Drama",actor,\N,Jackie Chan,"tt0127357,tt0089374,tt0120812,tt0266915"
4,nm0809727,tt6577606,tvEpisode,SAM SAM'S Silhouette,0,2009,\N,Animation,actor,"[""Wettabed Boss""]",Rob Smith,"tt0352248,tt0165710,tt0299658,tt0132245"


# Załadowanie nowych baz

   **W celu załadowania nowych danych należy:**


1. Pobrać token API:
  * Wejść w *zespół Aktorzy* na teams
   * Wybrać zakładkę *pliki*
    * Pobrać plik *kaggle.json* - API
2. Utworzyć plik na swoim dysku pod nazwą "Big Data"
3. Wrzucić pobrane API na swój dysk
4. Uruchomić kod :)




## Wczytajmy bazę oskarów: "Best Movies Watchlist"


In [14]:
from google.colab import drive
drive.mount('/content/gdrive')

#%cd /content/gdrive/MyDrive/Big Data/kaggle.json
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Big Data"

#from google.colab import drive
#drive.mount('/content/drive')
#random.seed(123456)
#np.set_printoptions(precision=6, suppress=True)

Mounted at /content/gdrive


In [15]:
! pip install kaggle
! mkdir ~/.kaggle #Utwórz katalog o nazwie „.kaggle”
! cp kaggle.json ~/.kaggle/ # Skopiuj plik „kaggle.json” do tego nowego katalogu
! chmod 600 ~/.kaggle/kaggle.json #Przydziel wymagane uprawnienia do tego pliku.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [16]:
#import pandas as pd

! kaggle datasets download unanimad/the-oscar-award #załadujmy dane
! unzip the-oscar-award.zip #rozpakujmy z zip'a
oscars=spark.read.option("header","true").csv('the_oscar_award.csv')
#oscars=pd.read_csv('the_oscar_award.csv')
oscars.show(5)


  0% 0.00/191k [00:00<?, ?B/s]
100% 191k/191k [00:00<00:00, 49.6MB/s]
Archive:  the-oscar-award.zip
  inflating: the_oscar_award.csv     
+---------+-------------+--------+--------+-------------------+----------------+------+
|year_film|year_ceremony|ceremony|category|               name|            film|winner|
+---------+-------------+--------+--------+-------------------+----------------+------+
|     1927|         1928|       1|   ACTOR|Richard Barthelmess|       The Noose| False|
|     1927|         1928|       1|   ACTOR|      Emil Jannings|The Last Command|  True|
|     1927|         1928|       1| ACTRESS|     Louise Dresser| A Ship Comes In| False|
|     1927|         1928|       1| ACTRESS|       Janet Gaynor|      7th Heaven|  True|
|     1927|         1928|       1| ACTRESS|     Gloria Swanson|  Sadie Thompson| False|
+---------+-------------+--------+--------+-------------------+----------------+------+
only showing top 5 rows



Main Data:
* primaryName	- aktor
* originalTitle - nazwa filmu

W nowej bazie oskarów:
* name - aktor
* film - film

## Wczytajmy bazę złotych globów: "Golden Globe Awards, 1944 - 2020"

In [17]:
! kaggle datasets download unanimad/golden-globe-awards #załadujmy dane
! unzip golden-globe-awards.zip #rozpakujmy z zip'a
globe=spark.read.option("header","true").csv('golden_globe_awards.csv')
globe.show(5)

  0% 0.00/117k [00:00<?, ?B/s]
100% 117k/117k [00:00<00:00, 46.5MB/s]
Archive:  golden-globe-awards.zip
  inflating: golden_globe_awards.csv  
+---------+----------+--------+--------------------+--------------------+--------------------+----+
|year_film|year_award|ceremony|            category|             nominee|                film| win|
+---------+----------+--------+--------------------+--------------------+--------------------+----+
|     1943|      1944|       1|Best Performance ...|      Katina Paxinou|For Whom The Bell...|True|
|     1943|      1944|       1|Best Performance ...|       Akim Tamiroff|For Whom The Bell...|True|
|     1943|      1944|       1|Best Director - M...|          Henry King|The Song Of Berna...|True|
|     1943|      1944|       1|             Picture|The Song Of Berna...|                null|True|
|     1943|      1944|       1|Actress In A Lead...|      Jennifer Jones|The Song Of Berna...|True|
+---------+----------+--------+--------------------+-----

## Wczytajmy bazę złotych globów: "Primetime Emmy Awards, 1949-2017"

In [18]:
! kaggle datasets download unanimad/emmy-awards #załadujmy dane
! unzip emmy-awards.zip #rozpakujmy z zip'a
emmy_awards_category=spark.read.option("header","true").csv('emmy_awards_categories.csv')
emmy_awards_category.show(5)


  0% 0.00/904k [00:00<?, ?B/s]
100% 904k/904k [00:00<00:00, 74.5MB/s]
Archive:  emmy-awards.zip
  inflating: emmy_awards_categories.csv  
  inflating: the_emmy_awards.csv     
+------------------+--------+----+
|              name|category|role|
+------------------+--------+----+
|     Comedy Series|  Comedy|main|
|        Lead Actor|  Comedy|main|
|      Lead Actress|  Comedy|main|
|  Supporting Actor|  Comedy|main|
|Supporting Actress|  Comedy|main|
+------------------+--------+----+
only showing top 5 rows



In [19]:
emmy_awards=spark.read.option("header","true").csv('the_emmy_awards.csv')
emmy_awards.show(5)

+---+----+--------------------+--------------------+--------------------+--------+--------------------+-----+
| id|year|            category|             nominee|               staff| company|            producer|  win|
+---+----+--------------------+--------------------+--------------------+--------+--------------------+-----+
|  1|2019|Outstanding Chara...|        The Simpsons|Hank Azaria, as M...|     FOX|Gracie Films in a...|False|
|  2|2019|Outstanding Chara...|          Family Guy|Alex Borstein, as...|     FOX|20th Century Fox ...|False|
|  3|2019|Outstanding Chara...|When You Wish Upo...|Eric Jacobson, as...|     HBO|Sesame Street Wor...|False|
|  4|2019|Outstanding Chara...|     F Is For Family|Kevin Michael Ric...| Netflix|Wild West Televis...|False|
|  5|2019|Outstanding Produ...| Escape At Dannemora|Mark Ricker, Prod...|Showtime|Red Hour, Busyhan...|False|
+---+----+--------------------+--------------------+--------------------+--------+--------------------+-----+
only showi

## Zaczytanie danych TMDB 

   informacje o aktorach i osobach pracujących przy produkcji filmów oraz budżetach, gatunkach, popularności, wytwórniach filmowych, oryginalnyh językach i tytułach


In [20]:
#!kaggle kernels init -p tmdb/tmdb-movie-metadata #wygenerować metadane
! kaggle datasets download tmdb/tmdb-movie-metadata #załadujmy dane
! unzip tmdb-movie-metadata.zip
tmdb_credits=spark.read.option("header","true").csv('tmdb_5000_credits.csv')
tmdb_movies=spark.read.option("header","true").csv('tmdb_5000_movies.csv')

  0% 0.00/8.89M [00:00<?, ?B/s]
100% 8.89M/8.89M [00:00<00:00, 122MB/s]
Archive:  tmdb-movie-metadata.zip
  inflating: tmdb_5000_credits.csv   
  inflating: tmdb_5000_movies.csv    


In [21]:
tmdb_credits.show(5)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
tmdb_movies.show(5)

# Połączmy dane na temat nagród z głównymi danymi

In [102]:
oscars_selected = oscars.filter((oscars.category.like('%ACTOR%')) | (oscars.category.like('%ACTRESS%')))
globe_selected = globe.filter((globe.category.like('%Actor%')) | (globe.category.like('%Actress%')))

In [103]:
oscars_selected = oscars.select(["year_ceremony", "category", "name", "film", "winner"])
globe_selected = globe.select(["year_award", "category", "nominee", "film", "win"])
#emmy_awards_selected = emmy_awards.select(["name", "category", "role"])

In [104]:
oscars_selected = oscars_selected.withColumnRenamed("year_ceremony","year_oscars").withColumnRenamed("category","category_oscars").withColumnRenamed("film","film_oscars").withColumnRenamed("winner","winner_oscars")

globe_selected = globe_selected.withColumnRenamed("year_award","year_globes").withColumnRenamed("category","category_globes").withColumnRenamed("film","film_globes").withColumnRenamed("win","win_globes")

#emmy_awards_selected = emmy_awards_selected.withColumnRenamed("category","category_emmy").withColumnRenamed("role","role_emmy")

In [105]:
# print("data dataframe size: ", (data.count(), len(data.columns)))
# print("oscars_selected dataframe size: ", (oscars_selected.count(), len(oscars_selected.columns)))
# print("globe_selected dataframe size: ", (globe_selected.count(), len(globe_selected.columns)))
#print("emmy_awards_selected dataframe size: ", (emmy_awards_selected.count(), len(emmy_awards_selected.columns)))
data = data.join(oscars_selected, data.primaryName == oscars_selected.name, "left")
# print("joined dataframe size: ", (data.count(), len(data.columns)))
data = data.join(globe_selected, data.primaryName == globe_selected.nominee, "left")
# print("joined dataframe size: ", (data.count(), len(data.columns)))
#data = data.join(emmy_awards_selected, data.primaryName == emmy_awards_selected.name, "left")
#print("joined dataframe size: ", (data.count(), len(data.columns)))

In [106]:
data = data.groupby('nconst').agg(first('tconst').alias("tconst"), first('titleType').alias("titleType"), first('originalTitle').alias("originalTitle"), first('isAdult').alias("isAdult"), first('startYear').alias("startYear"), first('endYear').alias("endYear"), first('genres').alias("genres"), first('category').alias("category"), first('characters').alias("characters"), first('primaryName').alias("primaryName"), first('knownForTitles').alias("knownForTitles"), collect_list('year_oscars').alias("year_oscars"), collect_list('category_oscars').alias("category_oscars"), collect_list('film_oscars').alias("film_oscars"), collect_list('winner_oscars').alias("winner_oscars"), collect_list('year_globes').alias("year_globes"), collect_list('category_globes').alias("category_globes"), collect_list('film_globes').alias("film_globes"), collect_list('win_globes').alias("win_globes"))

In [107]:
data.show()

+---------+--------------------+--------------------+--------------------+--------------------+---------+-------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|   nconst|              tconst|           titleType|       originalTitle|             isAdult|startYear|endYear|              genres|category|          characters|         primaryName|      knownForTitles|         year_oscars|     category_oscars|         film_oscars|       winner_oscars|         year_globes|     category_globes|         film_globes|          win_globes|
+---------+--------------------+--------------------+--------------------+--------------------+---------+-------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--

In [113]:
print(data.collect()[1])

Py4JJavaError: ignored

In [88]:
oscars_selected.select('name').distinct().count()
oscars_selected.select('name').count()

10395